<b>Import Dependencies

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo

from splinter import Browser

<h1>NASA Mars News

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.m2mars_db
collection = db.mars_news

In [4]:
# URL of page to be scraped
news_url = 'https://mars.nasa.gov/news/'
# Retrieve page with the requests module
response = requests.get(news_url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [5]:
title = soup.find('div', class_='content_title').find('a').text
news_title = title.replace("\n","")
news_title

'NASA Invites Students to Name Mars 2020 Rover'

In [6]:
paragraph = soup.find('div', class_='rollover_description_inner').text
news_paragraph = paragraph.replace("\n","")
news_paragraph

"Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover."

<h1>JPL Mars Space Images - Featured Image

In [7]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [8]:
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)

In [9]:
# dir(browser)

In [10]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

browser.click_link_by_partial_text('FULL IMAGE')

In [11]:
browser.find_by_css('a.fancybox-expand').first.click()

In [12]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [13]:
full_image = soup.find('div', class_='fancybox-inner').find('img')['src']

In [14]:
featured_image_url = f'https://www.jpl.nasa.gov{full_image}'
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17843_ip.jpg'

<h1>Mars Weather

In [16]:
# URL of page to be scraped
weather_url = 'https://twitter.com/marswxreport?lang=en'
# Retrieve page with the requests module
response = requests.get(weather_url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [64]:
tweets = soup.find_all('div', class_="js-tweet-text-container")

In [67]:
notweather = []
weathertweet = []

for tweet in tweets:
    weather_tweet = tweet.find('p', class_="tweet-text").text
    check_tweet = weather_tweet.startswith('InSight')
    if check_tweet is False:
        notweather.append(weather_tweet)
    elif check_tweet is True:
        weathertweet.append(weather_tweet)
        break
first_tweet = weathertweet[0]

'InSight sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)\nwinds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)\npressure at 7.50 hPapic.twitter.com/9YLawm67zS'

In [70]:
mars_weather = first_tweet.replace("\n"," ").replace("InSight sol","Sol")
mars_weather

'Sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF) winds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph) pressure at 7.50 hPapic.twitter.com/9YLawm67zS'

In [46]:
# soup

<h1>Mars Facts

<h1>Mars Hemispheres